In [3]:
import glob
import json
import re
import pickle
from pprint import pprint

def beautifyJSON(json_str, indent):
    return re.sub(r'\\u[0-9a-fA-F]{4}', lambda x:eval('"' + x.group() + '"'), 
                  json.dumps(json_str,indent=indent))

In [2]:
dictOfAllLines = {}

for fname in glob.glob("./*.js"):
    with open(fname) as f:
        raw =  '{' + ''.join(f.readlines()[2:])      
        quoted = re.sub(r"([a-zA-Z0-9_]+):","\"\\1\":", raw)
        quoted = re.sub(r"'","\"", quoted)
        btsLine = json.loads(quoted)
        dictOfAllLines = {**dictOfAllLines,**btsLine}

print(dictOfAllLines.keys())

with open('data.pickle', 'wb') as f:
    pickle.dump(dictOfAllLines, f, pickle.HIGHEST_PROTOCOL)

dict_keys(['arl', 'blue', 'brown', 'brt', 'btsEast', 'btsNorth', 'btsSouth', 'cyan', 'grey', 'orange', 'pink', 'purple', 'sralNorth', 'sralTransitMakkasan', 'sralTransitOrange', 'sralWest', 'yellow'])


In [9]:
with open('data.pickle', 'rb') as f:
    # The protocol version used is detected automatically, so we do not
    # have to specify it.
    dictOfAllLines = pickle.load(f)
    
selectedLines = dict([ (k, dictOfAllLines.get(k)) for k in ['arl','blue', 'brown',
                                                            'brt', 'btsEast', 'btsNorth',
                                                            'btsSouth', 'cyan', 'grey',
                                                            'orange', 'pink', 'purple',
                                                            'sralNorth', 'sralTransitMakkasan', 'sralTransitOrange',
                                                            'sralWest', 'yellow']])

In [22]:
def to8digits(s):
    return str(int(hashlib.sha256(s.encode('utf-8')).hexdigest(), 16) % 10**8)

import math, copy, hashlib
equator_miles =  69.172
lat_factor = 69.407
inch2pixel = 75
extra_scale = 2

cyNodeList = []
cyEdgeList = []

EST_train = 5
EST_short_walk = 5
EST_long_walk = 10

for k,v in selectedLines.items():

    line_name = k
    nodes = copy.deepcopy(v)

    id1 = id2 = None
    for el in nodes:
        
        ''' Construct nodes in Cytoscape's format '''

        lng_factorCal = lambda lat: math.cos(lat/180*math.pi)*equator_miles

        el['id'] = to8digits(line_name+','+el['name'])
        el['station_name'] = el['name']
        el['line_name'] = line_name
        lat = el['lat']
        lng = el['lng']

        del el['bg'], el['line'], el['name']

        cy_node = {}
        cy_node['data'] = el
        cy_node['position'] = {'x':lng*lng_factorCal(lat)*inch2pixel*extra_scale, 'y':-lat*lat_factor*inch2pixel*extra_scale}

        cyNodeList.append(cy_node)
        
        ''' Append edges '''
        
        id1 = id2
        id2 = el['id']
        
        if not id1 is None:
            cy_edge1 = {'data': {
                            'id': id1 + '_' +id2, 
                            'source': id1,
                            'target': id2,
                            'line_name': line_name,
                            'est_time': EST_train
                         }
                       }
            cy_edge2 = {'data': {
                            'id': id2 + '_' +id1, 
                            'source': id2,
                            'target': id1,
                            'line_name': line_name,
                            'est_time': EST_train
                         }
                       }
            cyEdgeList.append(cy_edge1)
            cyEdgeList.append(cy_edge2)
    

In [15]:
def addInterchange(line1, station1, line2, station2, est_time):
    
    id1 = to8digits('{},{}'.format(line1, station1))
    id2 = to8digits('{},{}'.format(line2, station2))
    
    cy_edge1 = {'data': {
                    'id': id1 + '__' +id2, 
                    'source': id1,
                    'target': id2,
                    'line_name': line1 + '__' + line2,
                    'est_time': est_time
                 }
               }
    cy_edge2 = {'data': {
                    'id': id2 + '__' +id1, 
                    'source': id2,
                    'target': id1,
                    'line_name': line2 + '__' + line1,
                    'est_time': est_time
                 }
               }
    cyEdgeList.append(cy_edge1)
    cyEdgeList.append(cy_edge2)
    
    return

# reference http://www.bangkokmetro.co.th/web/imgcontent/Image/map17latest.jpg


# เตาปูน mrt blue & เตาปูน mrt purple
addInterchange('blue','BL10-เตาปูน','purple','PP16-เตาปูน',EST_short_walk)
# จตุจักร mrt blue & หมอชิต bts sukhumvit
addInterchange('blue','BL13-จตุจักร','btsNorth','N8-หมอชิต',EST_short_walk)
# พญาไท bts sukhumvit & พญาไท airport rail link
addInterchange('btsNorth','N2-พญาไท','arl','A8-พญาไท',EST_short_walk)
# สุขุมวิท mrt blue & อโศก bts sukhumvit
addInterchange('blue','BL22-สุขุมวิท','btsNorth','E4-อโศก',EST_short_walk)
# มักกะสัน airport rail link & เพชรบุรี mrt blue
addInterchange('arl','A6-มักกะสัน','blue','BL21-เพชรบุรี',EST_long_walk)
# สยาม bts sukhumvit & สยาม bts silom
addInterchange('btsNorth','CEN-สยาม','btsSouth','CEN-สยาม',EST_short_walk)
# สีลม mrt blue & ศาลาแดง bts silom
addInterchange('blue','BL26-สีลม','btsSouth','S2-ศาลาแดง',EST_long_walk)
# สามยอด mrt purple & สนามไชย mrt blue
addInterchange('purple','PP23-สามยอด','blue','BL31-สนามไชย',EST_short_walk)
# บางหว้า mrt blue & บางหว้า bts silom
addInterchange('blue','BL34-บางหว้า','btsSouth','S12-บางหว้า',EST_short_walk)


In [16]:
jsonOutput = beautifyJSON(
    {'nodes': cyNodeList, 'edges': cyEdgeList}
    , 4)

with open('../data4JS.js','w') as f:
    #f.write('getGraph(' + jsonOutput + ');')
    f.write(jsonOutput)

In [27]:
pydict = str({'nodes': cyNodeList, 'edges': cyEdgeList})
unquoted = re.sub(r"'([a-zA-Z0-9_]+)':","\\1:", pydict)
nodeJSOutput = 'var graphP = {};\nexports.graphP = graphP;'.format(unquoted)

with open('../data4NodeJS.js','w') as f:
    f.write(nodeJSOutput)